# Data Analysis - Survival Analysis

Data from: https://xenabrowser.net/datapages/?dataset=GDC-PANCAN.htseq_fpkm-uq.tsv&host=https%3A%2F%2Fgdc.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

### TCGA Barcodes
The column headers are TCGA barcodes:
* In the format of: `project-tissuesourcesite-participant-sample|vial-portion|analyte-plate-center`
* https://docs.gdc.cancer.gov/Encyclopedia/pages/TCGA_Barcode/
* https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/tissue-source-site-codes

In [ ]:
# Installations
# !pip install kaplanmeier
# !pip install gseapy

In [1]:
# Imports
import pandas as pd
import gseapy as gp

In [3]:
# Read in the RNA matrix
df = pd.read_parquet('./data/GDC-PANCAN.htseq_fpkm-uq.parquet')
display(df.head())

,xena_sample,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,TCGA-OR-A5JR-01A,TCGA-OR-A5KU-01A,TCGA-OR-A5LS-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5JQ-01A,...,TARGET-50-PAJMKJ-01A,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A
0,ENSG00000242268.2,0.000000,0.000000,0.000000,0.000000,9.486642,0.000000,0.000000,0.000000,0.000000,...,11.700035,10.041859,13.398458,0.000000,10.617230,11.933609,14.140998,11.659218,10.662028,12.878131
1,ENSG00000270112.3,10.689655,14.408626,14.022621,11.291444,10.221394,12.423503,12.830424,12.758888,11.547426,...,9.267574,12.513257,10.501003,10.452072,10.625798,8.310776,7.131909,7.678919,10.134942,11.116645
2,ENSG00000167578.15,18.536987,18.684183,17.334107,19.713465,16.761630,17.762472,18.114361,19.068519,17.474470,...,15.541309,16.684341,15.905948,16.991286,15.066989,13.953978,15.969451,14.607776,14.387707,15.886538
3,ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000078237.5,17.847476,18.227483,17.287893,16.722624,17.157762,17.001996,18.648729,18.076084,15.817248,...,15.377730,16.438256,16.733394,16.149538,16.277784,15.673957,16.305087,15.916629,15.850915,16.188748


### Align RNA / OS / Phenotype samples

---

In [4]:
# Read in ID/Gene Mapping file
mapping_df = pd.read_csv('./data/gencode.v22.annotation.gene.probeMap', sep='\t')
display(mapping_df.head())

,id,gene,chrom,chromStart,chromEnd,strand
0,ENSG00000223972.5,DDX11L1,chr1,11869,14409,+
1,ENSG00000227232.5,WASH7P,chr1,14404,29570,-
2,ENSG00000278267.1,MIR6859-3,chr1,17369,17436,-
3,ENSG00000243485.3,RP11-34P13.3,chr1,29554,31109,+
4,ENSG00000274890.1,MIR1302-9,chr1,30366,30503,+


In [5]:
# Check to see if the mapping file and the rna matrix file have the same id names
mapping_df.rename(columns={'id': 'xena_sample'}, inplace=True)
merged_df = pd.merge(mapping_df, df, on='xena_sample', how='outer', indicator=True)

# Check matching status
# Filter rows that do not have 'both' in the '_merge' column
non_matching_rows = merged_df[merged_df['_merge'] != 'both']

# Print the non-matching rows
print(non_matching_rows)

Empty DataFrame
Columns: [xena_sample, gene, chrom, chromStart, chromEnd, strand, TCGA-OR-A5JP-01A, TCGA-OR-A5JE-01A, TCGA-OR-A5JG-01A, TCGA-OR-A5L9-01A, TCGA-OR-A5JR-01A, TCGA-OR-A5KU-01A, TCGA-OR-A5LS-01A, TCGA-OR-A5J7-01A, TCGA-OR-A5JQ-01A, TCGA-OR-A5JS-01A, TCGA-OR-A5JL-01A, TCGA-OR-A5LC-01A, TCGA-OR-A5K2-01A, TCGA-P6-A5OG-01A, TCGA-OR-A5JW-01A, TCGA-OR-A5JZ-01A, TCGA-OR-A5J8-01A, TCGA-OR-A5K5-01A, TCGA-OR-A5KV-01A, TCGA-OR-A5L4-01A, TCGA-OR-A5KX-01A, TCGA-OR-A5K1-01A, TCGA-OR-A5JO-01A, TCGA-OR-A5LG-01A, TCGA-OR-A5LO-01A, TCGA-OR-A5JB-01A, TCGA-OR-A5JV-01A, TCGA-OR-A5LJ-01A, TCGA-OR-A5LA-01A, TCGA-OR-A5KY-01A, TCGA-OR-A5KO-01A, TCGA-OR-A5L6-01A, TCGA-OR-A5KZ-01A, TCGA-OR-A5J5-01A, TCGA-OR-A5LB-01A, TCGA-OR-A5LT-01A, TCGA-OR-A5LD-01A, TCGA-OR-A5J2-01A, TCGA-OR-A5LE-01A, TCGA-OR-A5K4-01A, TCGA-OR-A5K6-01A, TCGA-OR-A5JY-01A, TCGA-OR-A5JT-01A, TCGA-OR-A5KW-01A, TCGA-PK-A5H8-01A, TCGA-OR-A5JX-01A, TCGA-OR-A5LK-01A, TCGA-P6-A5OF-01A, TCGA-OR-A5JM-01A, TCGA-OR-A5JI-01A, TCGA-OR-A5JC-01A, 

In [6]:
display(df.head())
print(df.info())

,xena_sample,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,TCGA-OR-A5JR-01A,TCGA-OR-A5KU-01A,TCGA-OR-A5LS-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5JQ-01A,...,TARGET-50-PAJMKJ-01A,TARGET-50-CAAAAQ-11A,TARGET-50-PAKSCC-01A,TARGET-50-PAJNSL-11A,TARGET-50-PAJPAU-01A,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A
0,ENSG00000242268.2,0.000000,0.000000,0.000000,0.000000,9.486642,0.000000,0.000000,0.000000,0.000000,...,11.700035,10.041859,13.398458,0.000000,10.617230,11.933609,14.140998,11.659218,10.662028,12.878131
1,ENSG00000270112.3,10.689655,14.408626,14.022621,11.291444,10.221394,12.423503,12.830424,12.758888,11.547426,...,9.267574,12.513257,10.501003,10.452072,10.625798,8.310776,7.131909,7.678919,10.134942,11.116645
2,ENSG00000167578.15,18.536987,18.684183,17.334107,19.713465,16.761630,17.762472,18.114361,19.068519,17.474470,...,15.541309,16.684341,15.905948,16.991286,15.066989,13.953978,15.969451,14.607776,14.387707,15.886538
3,ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000078237.5,17.847476,18.227483,17.287893,16.722624,17.157762,17.001996,18.648729,18.076084,15.817248,...,15.377730,16.438256,16.733394,16.149538,16.277784,15.673957,16.305087,15.916629,15.850915,16.188748


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60483 entries, 0 to 60482
Columns: 11769 entries, xena_sample to TARGET-50-PALFRD-01A
dtypes: float64(11768), object(1)
memory usage: 5.3+ GB
None


In [7]:
# Merge df with mapping_df on the 'id' column
df_merged = pd.merge(df, mapping_df, on='xena_sample', how='left')

In [8]:
# Print the updated df2 DataFrame
display(df_merged.head())

,xena_sample,TCGA-OR-A5JP-01A,TCGA-OR-A5JE-01A,TCGA-OR-A5JG-01A,TCGA-OR-A5L9-01A,TCGA-OR-A5JR-01A,TCGA-OR-A5KU-01A,TCGA-OR-A5LS-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5JQ-01A,...,TARGET-50-PAJNZU-01A,TARGET-50-PAJNNR-01A,TARGET-50-PAJNTJ-02A,TARGET-50-PAECJB-01A,TARGET-50-PALFRD-01A,gene,chrom,chromStart,chromEnd,strand
0,ENSG00000242268.2,0.000000,0.000000,0.000000,0.000000,9.486642,0.000000,0.000000,0.000000,0.000000,...,11.933609,14.140998,11.659218,10.662028,12.878131,RP11-368I23.2,chr3,168903366,168921996,+
1,ENSG00000270112.3,10.689655,14.408626,14.022621,11.291444,10.221394,12.423503,12.830424,12.758888,11.547426,...,8.310776,7.131909,7.678919,10.134942,11.116645,RP11-742D12.2,chr18,46756487,46764408,+
2,ENSG00000167578.15,18.536987,18.684183,17.334107,19.713465,16.761630,17.762472,18.114361,19.068519,17.474470,...,13.953978,15.969451,14.607776,14.387707,15.886538,RAB4B,chr19,40778216,40796944,+
3,ENSG00000273842.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,AC104183.2,chr3,21382478,21382542,+
4,ENSG00000078237.5,17.847476,18.227483,17.287893,16.722624,17.157762,17.001996,18.648729,18.076084,15.817248,...,15.673957,16.305087,15.916629,15.850915,16.188748,C12orf5,chr12,4321205,4354593,+


In [9]:
# Read in the basic phenotype data
df_basic_phenotype = pd.read_parquet('./data/GDC-PANCAN.basic_phenotype.parquet')

In [10]:
display(df_basic_phenotype.head())
print(df_basic_phenotype.info())

,sample,program,sample_type_id,sample_type,project_id,Age at Diagnosis in Years,Gender
0,TCGA-69-7978-01A,TCGA,1,Primary Tumor,TCGA-LUAD,59.000000,Male
1,TCGA-AR-A24Z-01A,TCGA,1,Primary Tumor,TCGA-BRCA,57.000000,Female
2,TCGA-D1-A103-01A,TCGA,1,Primary Tumor,TCGA-UCEC,87.000000,Female
3,TARGET-20-PASRLS-09A,TARGET,9,Primary Blood Derived Cancer - Bone Marrow,TARGET-AML,0.816438,Female
4,TARGET-20-PASARK-14A,TARGET,14,Bone Marrow Normal,TARGET-AML,15.520548,Male


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19188 entries, 0 to 19187
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   sample                     19188 non-null  object 
 1   program                    19188 non-null  object 
 2   sample_type_id             19188 non-null  int64  
 3   sample_type                19117 non-null  object 
 4   project_id                 18954 non-null  object 
 5   Age at Diagnosis in Years  18677 non-null  float64
 6   Gender                     18738 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.0+ MB
None


In [11]:
# Read in the survival phenotype data
df_survival_phenotype = pd.read_parquet('./data/GDC-PANCAN.survival.parquet')

In [12]:
display(df_survival_phenotype.head())
print(df_survival_phenotype.info())

,sample,OS,_PATIENT,OS.time
0,TCGA-OR-A5KZ-01A,1,TCGA-OR-A5KZ,125
1,TCGA-OR-A5LC-01A,1,TCGA-OR-A5LC,159
2,TCGA-P6-A5OF-01A,1,TCGA-P6-A5OF,207
3,TCGA-OR-A5JU-01A,1,TCGA-OR-A5JU,289
4,TCGA-OR-A5K9-11A,1,TCGA-OR-A5K9,344


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18492 entries, 0 to 18491
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sample    18492 non-null  object
 1   OS        18492 non-null  int64 
 2   _PATIENT  18492 non-null  object
 3   OS.time   18492 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 578.0+ KB
None


In [13]:
# Create a list of all samples
samples_rna = list(df_merged.columns)
samples_pheno = list(df_basic_phenotype['sample'].values)
samples_survival = list(df_survival_phenotype['sample'].values)

In [14]:
# Find all common samples in all three lists
common_samples = list(set(samples_rna) & set(samples_pheno) & set(samples_survival))

In [15]:
# Subset and reorder all three datasets by common_samples
# Filter merged_df by columns in common_samples
df_merged_filtered = df_merged[common_samples]
df_basic_phenotype_filtered = df_basic_phenotype[df_basic_phenotype['sample'].isin(common_samples)]
df_survival_phenotype_filtered = df_survival_phenotype[df_survival_phenotype['sample'].isin(common_samples)]

In [16]:
display(df_merged_filtered.head())
display(df_basic_phenotype_filtered.head())
display(df_survival_phenotype_filtered.head())

,TCGA-BS-A0V4-01A,TCGA-BT-A42E-01A,TCGA-FV-A4ZQ-01A,TCGA-BT-A20T-01A,TCGA-G9-6351-11A,TCGA-A4-A57E-11A,TCGA-CC-A8HU-01A,TCGA-ZF-AA56-01A,TCGA-39-5037-01A,TCGA-BH-A1F6-01A,...,TCGA-BH-A0HU-01A,TCGA-FM-8000-01A,TCGA-P4-A5ED-11A,TCGA-ET-A2MX-01A,TCGA-XK-AAJU-01A,TCGA-K1-A6RV-01A,TCGA-EK-A2H1-01A,TCGA-A6-3810-01A,TCGA-AJ-A3EJ-01A,TCGA-HU-A4GY-01A
0,0.000000,0.000000,0.000000,0.000000,0.000000,10.846438,0.000000,0.000000,0.000000,0.000000,...,8.950333,0.000000,12.070347,14.050258,9.522254,9.671725,0.000000,0.000000,9.715727,8.861777
1,0.000000,0.000000,0.000000,0.000000,0.000000,11.950808,6.942390,0.000000,0.000000,0.000000,...,6.118916,0.000000,10.114991,0.000000,6.684973,8.409759,7.438836,8.353073,0.000000,7.020421
2,18.856675,16.983636,16.113740,16.968398,16.844261,16.007466,16.476247,16.473858,16.260159,15.251752,...,16.001751,18.079729,15.722215,16.178899,16.173226,16.322889,16.677535,15.618298,16.287217,16.719516
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,13.522942,17.818713,15.007085,15.032644,15.619894,15.310189,13.937949,16.784777,16.912745,16.776239,...,16.251805,17.616219,16.166940,16.591534,16.466390,17.020745,17.938852,16.999805,16.434509,16.812989


,sample,program,sample_type_id,sample_type,project_id,Age at Diagnosis in Years,Gender
0,TCGA-69-7978-01A,TCGA,1,Primary Tumor,TCGA-LUAD,59.0,Male
1,TCGA-AR-A24Z-01A,TCGA,1,Primary Tumor,TCGA-BRCA,57.0,Female
2,TCGA-D1-A103-01A,TCGA,1,Primary Tumor,TCGA-UCEC,87.0,Female
5,TCGA-24-1435-01A,TCGA,1,Primary Tumor,TCGA-OV,57.0,Female
7,TCGA-63-A5MB-01A,TCGA,1,Primary Tumor,TCGA-LUSC,62.0,Male


,sample,OS,_PATIENT,OS.time
0,TCGA-OR-A5KZ-01A,1,TCGA-OR-A5KZ,125
1,TCGA-OR-A5LC-01A,1,TCGA-OR-A5LC,159
2,TCGA-P6-A5OF-01A,1,TCGA-P6-A5OF,207
5,TCGA-OR-A5K9-01A,1,TCGA-OR-A5K9,344
6,TCGA-OR-A5J5-01A,1,TCGA-OR-A5J5,365


In [17]:
# Reorder phenotype and survival dataframes to match the columns of the rna matrix
column_order = list(df_merged_filtered.columns)
df_basic_phenotype_filtered_ordered = df_basic_phenotype_filtered.set_index('sample').loc[column_order].reset_index()
df_survival_phenotype_filtered_ordered = df_survival_phenotype_filtered.set_index('sample').loc[column_order].reset_index()
# display(df_basic_phenotype_filtered_ordered)
# display(df_survival_phenotype_filtered_ordered)
# display(df_merged_filtered)

### GSVA: gene set variation analysis

---

Ref: https://gseapy.readthedocs.io/en/latest/gseapy_example.html#GSVA-example   
Ref: https://bioconductor.statistik.tu-dortmund.de/packages/3.16/bioc/vignettes/GSVA/inst/doc/GSVA.html#1_Quick_start   
Ref: https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#GMT:_Gene_Matrix_Transposed_file_format_.28.2A.gmt.29   
Ref: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html

In [ ]:
# TESTING - Write fake *.gmt file for testing
gene_sets_file = pd.DataFrame(data=['EGFR', 'FGR'])
gene_sets_file.to_csv('./data/testing.gmt', index=False)

In [ ]:
# TODO: Locate or create a Gene Matrix Transposed file (*.gmt) containing the gene sets to use for the GSVA

# Locate gene sets file
# gene_sets_file = '.gmt'
gene_sets_file = pd.read_csv('./data/testing.gmt', index_col=False)

# Run GSVA
es = gp.gsva(data=df_merged_filtered, # use the filtered rna matrix
             gene_sets=gene_sets_file,
             outdir=None,  # Avoid saving output to disk
             format='tsv')

# Since data is large, consider using the optional parameters to manage memory
# For example, set 'min_size' and 'max_size' to limit gene sets sizes

# Check the results
result = es.res2d.pivot(index='Term', columns='Name', values='ES')

# Display the first few rows of the results
print(result.head())

,0
0,EGFR
1,FGR


### KM Plot

---

OS Values:
* 1 = `deceased`
* 0 = `living`

Ref: https://docs.cbioportal.org/user-guide/faq/#what-is-the-meaning-of-os_status--os_months-and-pfs_status--pfs_months   
Ref: https://erdogant.github.io/kaplanmeier/pages/html/Examples.html

In [62]:
import kaplanmeier as km

In [ ]:
# EXAMPLE: https://erdogant.github.io/kaplanmeier/pages/html/Examples.html
time_event = df_survival_phenotype_filtered_ordered['OS.time']
censoring = df_survival_phenotype_filtered_ordered['OS']
y = df['group']

print(df)
#       time  Died  group
# 0     485     0      1
# 1     526     1      2
# 2     588     1      2
# 3     997     0      1
# 4     426     1      1
# ..    ...   ...    ...
# 175   183     0      1
# 176  3196     0      1
# 177   457     1      2
# 178  2100     1      1
# 179   376     0      1
#
# [180 rows x 3 columns]

# Compute Survival
results = km.fit(time_event, censoring, y)

# Plot
km.plot(results)

### Transfer CSV files to Parquet

---

In [20]:
# Read in the csv into a df
# df_transfer = pd.read_csv('./data/GDC-PANCAN.htseq_fpkm-uq.tsv', sep='\t')
# df_survival_transfer = pd.read_csv('./data/GDC-PANCAN.survival.tsv', sep='\t')
# df_basic_pheno_transfer = pd.read_csv('./data/GDC-PANCAN.basic_phenotype.tsv', sep='\t')

# Transform csv's into parquet files
# df_transfer.to_parquet('./data/GDC-PANCAN.htseq_fpkm-uq.parquet', compression=None)
# df_survival_transfer.to_parquet('./data/GDC-PANCAN.survival.parquet', compression=None)
# df_basic_pheno_transfer.to_parquet('./data/GDC-PANCAN.basic_phenotype.parquet', compression=None)